Import Dependencies

In [1]:
import time
import requests, pandas as pd
from datetime import datetime, timedelta, timezone, time
from datetime import datetime, timezone, time
import os
from pathlib import Path
import meteomatics.api as api
import matplotlib.pyplot as plt
import pytz

Présentation Location de la Bouée 51101

In [2]:
"""
import folium

Tournai = (50.6, 3.3833)
Buoy_51101 = (24.359, -162.081)
# Définir les coordonnées (latitude, longitude)
coordinates = [Buoy_51101, Tournai]

# Créer une carte centrée sur la première paire de coordonnées
map = folium.Map(location=coordinates[0], zoom_start=3)

# Ajouter des marqueurs pour chaque paire de coordonnées
for lat, lon in coordinates:
    folium.Marker(location=[lat, lon]).add_to(map)

# Enregistrer la carte dans un fichier HTML
map.save("map.html")

"""

'\nimport folium\n\nTournai = (50.6, 3.3833)\nBuoy_51101 = (24.359, -162.081)\n# Définir les coordonnées (latitude, longitude)\ncoordinates = [Buoy_51101, Tournai]\n\n# Créer une carte centrée sur la première paire de coordonnées\nmap = folium.Map(location=coordinates[0], zoom_start=3)\n\n# Ajouter des marqueurs pour chaque paire de coordonnées\nfor lat, lon in coordinates:\n    folium.Marker(location=[lat, lon]).add_to(map)\n\n# Enregistrer la carte dans un fichier HTML\nmap.save("map.html")\n\n'

Get Credentials

In [3]:
def read_credentials():
    with open('Constants.py', 'r') as credentials:
        content = credentials.read()
    lines = content.split('\n')
    lines = [line.strip() for line in lines]

    # Iterate through each line to find the values
    for line in lines:
        if line.startswith('METEO_USERNAME'):
            meteo_username = line.split('=')[1].strip().strip("'\"")
        elif line.startswith('METEO_PASSWORD'):
            meteo_password = line.split('=')[1].strip().strip("'\"")
    
    return meteo_username, meteo_password

Gather Last 24h Climate Data for Hawaii 51101 Station Buoy Zone by 10 minutes intervals

In [4]:
# Définir les paramètres
parameters = [
        'wind_speed_10m:ms',	
        'wind_dir_10m:d',
        'wind_gusts_10m_1h:ms',
        't_2m:C',
        't_max_2m_24h:C',
        't_min_2m_24h:C',
        'msl_pressure:hPa',
        'precip_1h:mm'
    ]

# Définir les coordonnées
coordinates = [(24.359, -162.081)]

# Définir le modèle
model = 'mix'
# Début URL de l'API
DEFAULT_API_BASE_URL = "https://api.meteomatics.com"
enddate = datetime.now(timezone.utc).replace(minute=0, second=0, microsecond=0)
startdate = enddate - timedelta(days=1)
interval = timedelta(minutes=10)

# Fonction de requête API Météo
def meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval):

    # Appel de la fonction read_credentials pour obtenir les informations d'identification
    meteo_username, meteo_password = read_credentials()
    
    # Récupérer les données
    meteo_df = api.query_time_series(coordinates, startdate, enddate, interval, parameters, meteo_username, meteo_password, model=model)
    meteo_df.reset_index(inplace=True)
    return meteo_df

meteo_df=meteo_api_request(parameters, coordinates, model, DEFAULT_API_BASE_URL, startdate, enddate, interval)
meteo_df.head() 

,lat,lon,validdate,wind_speed_10m:ms,wind_dir_10m:d,wind_gusts_10m_1h:ms,t_2m:C,t_max_2m_24h:C,t_min_2m_24h:C,msl_pressure:hPa,precip_1h:mm
0,24.359,-162.081,2024-05-20 14:00:00+00:00,5.0,161.7,5.0,24.0,24.5,21.8,1015.8,0.81
1,24.359,-162.081,2024-05-20 14:10:00+00:00,5.0,159.7,5.0,24.0,24.5,21.8,1015.9,0.63
2,24.359,-162.081,2024-05-20 14:20:00+00:00,5.0,157.3,5.0,24.0,24.5,21.8,1015.9,0.63
3,24.359,-162.081,2024-05-20 14:30:00+00:00,6.0,154.4,6.0,24.1,24.5,21.8,1015.9,0.63
4,24.359,-162.081,2024-05-20 14:40:00+00:00,6.0,151.0,6.0,23.9,24.5,21.8,1016.0,0.63


In [5]:
meteo_df.rename(columns={'validdate': 'Datetime', 'wind_speed_10m:ms':'wind speed (ms)','wind_dir_10m:d':'wind dir (°)',
                          'wind_gusts_10m_1h:ms':'wind_gusts 1h (ms)','t_2m:C':'T° (C°)','t_max_2m_24h:C':'T°_max 24h (C°)',
                          't_min_2m_24h:C':'T° min 24h (°)', 'msl_pressure:hPa': 'sea level pressure (hPa)','precip_1h:mm':'Precip 1h (mm)'}, inplace=True)
meteo_df = meteo_df.drop(columns=['lat','lon'])
meteo_df.head()

,Datetime,wind speed (ms),wind dir (°),wind_gusts 1h (ms),T° (C°),T°_max 24h (C°),T° min 24h (°),sea level pressure (hPa),Precip 1h (mm)
0,2024-05-20 14:00:00+00:00,5.0,161.7,5.0,24.0,24.5,21.8,1015.8,0.81
1,2024-05-20 14:10:00+00:00,5.0,159.7,5.0,24.0,24.5,21.8,1015.9,0.63
2,2024-05-20 14:20:00+00:00,5.0,157.3,5.0,24.0,24.5,21.8,1015.9,0.63
3,2024-05-20 14:30:00+00:00,6.0,154.4,6.0,24.1,24.5,21.8,1015.9,0.63
4,2024-05-20 14:40:00+00:00,6.0,151.0,6.0,23.9,24.5,21.8,1016.0,0.63


In [6]:
meteo_df.dtypes

Datetime                    datetime64[ns, UTC]
wind speed (ms)                         float64
wind dir (°)                            float64
wind_gusts 1h (ms)                      float64
T° (C°)                                 float64
T°_max 24h (C°)                         float64
T° min 24h (°)                          float64
sea level pressure (hPa)                float64
Precip 1h (mm)                          float64
dtype: object

Get Google Big Query Credentials And Send the Data

In [7]:
meteo_df.columns

Index(['Datetime', 'wind speed (ms)', 'wind dir (°)', 'wind_gusts 1h (ms)',
       'T° (C°)', 'T°_max 24h (C°)', 'T° min 24h (°)',
       'sea level pressure (hPa)', 'Precip 1h (mm)'],
      dtype='object')

In [8]:
meteo_df_final= meteo_df
meteo_df_final.to_csv('df_meteo_final.csv', index=False)